In [2]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import linear_model
from sklearn import datasets
from yellowbrick.target import FeatureCorrelation
import seaborn as sns
from datetime import datetime

In [3]:
df_1 = pd.read_csv(".\\audiobooks.csv")
df_2 = pd.read_csv(".\\audiobooks_2.csv")
df = df_1.merge(df_2, on='audiobook_id').set_index('audiobook_id').drop("Unnamed: 0_x",axis=1).drop("Unnamed: 0_y",axis=1).dropna()
# df = df[['price', 'release_date', 'length', 'rating_overall', 'rating_performance', 'rating_story']].dropna()

def convert_time(time_str):
    year = time_str[-2:]
    time_str = time_str[:-2] + '20' + year
    date_time = datetime.strptime(time_str, '%m-%d-%Y')
    return date_time.timestamp()

def convert_duration(length):
    # 9 hrs and 15 mins
    
    if 'hrs' in length:
        hours = length.split('hrs')[0].strip() # 9
    else:
        hours = length.split('hr')[0].strip() # 9
    if 'mins' in length:
        minutes = length[:-5].split(' ')[-1] # 15
        time = int(hours)*60 + int(minutes)
    else:
        time = int(hours)*60
    return time

df['release_date'] = df['release_date'].apply(convert_time)
df['length'] = df['length'].apply(convert_duration)
df['price'] = df['price'].apply(lambda s : float(s[1:]))
df

,title,author,narrator,summary,audiobook_link,length,release_date,price,categories,rating_overall,rating_performance,rating_story
audiobook_id,,,,,,,,,,,,
B09RQ4HRHN,"Friends, Lovers, and the Big Terrible Thing",Matthew Perry,Matthew Perry,The beloved star of Friends takes us behind th...,/pd/Friends-Lovers-and-the-Big-Terrible-Thing-...,529,1.667257e+09,25.51,Arts & Entertainment,4.6,4.7,4.6
B09SBP7MWN,It Starts with Us,Colleen Hoover,"Colin Donnell, Olivia Song","Lily and her ex-husband, Ryle, have just settl...",/pd/It-Starts-with-Us-Audiobook/B09SBP7MWN,521,1.666044e+09,23.62,Romance,4.7,4.8,4.6
B09R62PV4B,Fairy Tale,Stephen King,"Seth Numrich, Stephen King",Charlie Reade looks like a regular high school...,/pd/Fairy-Tale-Audiobook/B09R62PV4B,1446,1.662415e+09,26.90,"Mystery, Thriller & Suspense",4.8,4.9,4.7
B09VHWHJS2,I&apos;m Glad My Mom Died,Jennette McCurdy,Jennette McCurdy,Jennette McCurdy was six years old when she ha...,/pd/Im-Glad-My-Mom-Died-Audiobook/B09VHWHJS2,386,1.659996e+09,13.99,Biographies & Memoirs,4.9,4.9,4.9
1524779261,Atomic Habits,James Clear,James Clear,"No matter your goals, Atomic Habits offers a p...",/pd/Atomic-Habits-Audiobook/1524779261,335,1.539641e+09,16.72,Business & Careers,4.8,4.8,4.8
...,...,...,...,...,...,...,...,...,...,...,...,...
B0BK2RGQBT,Revenge,Michael Cohen,Keith Szarabajka,When Michael Cohen's secret payoff to porn sta...,/pd/Revenge-Audiobook/B0BK2RGQBT,473,1.666303e+09,19.95,Biographies & Memoirs,4.5,4.4,4.4
B09V1X64YV,Match Game,Craig Alanson,R.C. Bray,"For years, the ancient alien AI known as Skipp...",/pd/Match-Game-Audiobook/B09V1X64YV,1220,1.654553e+09,59.99,Science Fiction & Fantasy,4.9,5.0,4.8
0062963708,The Dutch House,Ann Patchett,Tom Hanks,"At the end of the Second World War, Cyril Conr...",/pd/The-Dutch-House-Audiobook/0062963708,593,1.569276e+09,24.90,Literature & Fiction,4.6,4.9,4.6


In [6]:
def root(word):
    word = ''.join(i for i in word if i.isalpha())
    if not word:
        return '$'
    end = len(word) - 1
    if word[end] in 'ds':
        end -= 1
    while end >= 0 and word[end] in 'aeiou':
        end -= 1
    return word[:end + 1]

def lexical_roots(string):
    return pd.Series(sorted({
        root(word) for word in string.strip("'.,").replace('-', ' ').lower().split() if word
    }))

In [7]:
lexical_roots('They say you can never go home again, and for Persephone Fraser, ever since she made the biggest mistake of her life a decade ago, that has felt too true. Instead of glittering summers on the lakeshore of her childhood, she spends them in a stylish apartment in the city, going out with friends, and keeping everyone a safe distance from her heart. Until she receives the call that sends her racing back to Barry’s Bay and into the orbit of Sam Florek—the man she never thought she’d have to live without.')

0              
1            ag
2         again
3            an
4     apartment
        ...    
64           tr
65        until
66         with
67      without
68            y
Length: 69, dtype: object

In [8]:
cat_df = df[['categories', 'summary']].dropna().reset_index()
category_prob = cat_df.groupby('categories').size() / len(df)
category_prob.sort_values()

categories
Home & Garden                                      0.001307
Erotica                                            0.001307
None                                               0.002614
Science & Engineering                              0.003922
Comedy & Humor                                     0.003922
Education & Learning                               0.003922
LGBTQ+                                             0.006536
Arts & Entertainment                               0.009150
Religion & Spirituality                            0.015686
Money & Finance                                    0.016993
Children\'s Audiobooks                             0.019608
History                                            0.020915
Politics & Social Sciences                         0.023529
Teen & Young Adult                                 0.043137
Romance                                            0.057516
Relationships, Parenting & Personal Development    0.060131
Health & Wellness            

In [9]:
cat_df

,audiobook_id,categories,summary
0,B09RQ4HRHN,Arts & Entertainment,The beloved star of Friends takes us behind th...
1,B09SBP7MWN,Romance,"Lily and her ex-husband, Ryle, have just settl..."
2,B09R62PV4B,"Mystery, Thriller & Suspense",Charlie Reade looks like a regular high school...
3,B09VHWHJS2,Biographies & Memoirs,Jennette McCurdy was six years old when she ha...
4,1524779261,Business & Careers,"No matter your goals, Atomic Habits offers a p..."
...,...,...,...
760,B0BK2RGQBT,Biographies & Memoirs,When Michael Cohen's secret payoff to porn sta...
761,B09V1X64YV,Science Fiction & Fantasy,"For years, the ancient alien AI known as Skipp..."
762,0062963708,Literature & Fiction,"At the end of the Second World War, Cyril Conr..."
763,1250752337,"Mystery, Thriller & Suspense","One night, Molly Clarke walked away from her l..."


In [10]:
category_prob = cat_df.groupby('categories').size() / len(cat_df)
category_prob.sort_values()
df.summary

audiobook_id
B09RQ4HRHN    The beloved star of Friends takes us behind th...
B09SBP7MWN    Lily and her ex-husband, Ryle, have just settl...
B09R62PV4B    Charlie Reade looks like a regular high school...
B09VHWHJS2    Jennette McCurdy was six years old when she ha...
1524779261    No matter your goals, Atomic Habits offers a p...
                                    ...                        
B0BK2RGQBT    When Michael Cohen's secret payoff to porn sta...
B09V1X64YV    For years, the ancient alien AI known as Skipp...
0062963708    At the end of the Second World War, Cyril Conr...
1250752337    One night, Molly Clarke walked away from her l...
B09LVXS3L4    The Real Anthony Fauci details how Fauci, Gate...
Name: summary, Length: 765, dtype: object

In [11]:
df.summary.apply(
    lexical_roots
).reset_index()

,audiobook_id,0,1,2,3,4,5,6,7,8,...,77,78,79,80,81,82,83,84,85,86
0,B09RQ4HRHN,,about,addiction,all,an,awar,battl,behin,belov,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,B09SBP7MWN,,after,again,an,ar,ask,atl,being,bump,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,B09R62PV4B,,$,accident,aging,an,at,backyar,baseball,big,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,B09VHWHJS2,,acting,ag,all,along,an,anything,ar,at,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1524779261,,again,an,atomic,b,becaus,behavior,break,but,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760,B0BK2RGQBT,,behalf,by,cam,charg,cohen,daniel,decad,donal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
761,B09V1X64YV,,$,abl,after,alien,als,an,ancient,another,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
762,0062963708,,acerbic,an,ar,assur,at,begin,brilliantly,busines,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
763,1250752337,,abandon,all,an,at,away,b,back,behin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
book_roots = df.summary.apply(
    lexical_roots
).stack(
).reset_index(
    level='audiobook_id'
).rename(columns={
    'audiobook_id': 'audiobook_id',
    0: 'root',
})
book_roots

,audiobook_id,root
0,B09RQ4HRHN,
1,B09RQ4HRHN,about
2,B09RQ4HRHN,addiction
3,B09RQ4HRHN,all
4,B09RQ4HRHN,an
...,...,...
38,B09LVXS3L4,th
39,B09LVXS3L4,their
40,B09LVXS3L4,us
41,B09LVXS3L4,virulenc


In [13]:
cat_roots = pd.merge(
    book_roots,
    cat_df
)[['root', 'categories']]
cat_roots

,root,categories
0,,Arts & Entertainment
1,about,Arts & Entertainment
2,addiction,Arts & Entertainment
3,all,Arts & Entertainment
4,an,Arts & Entertainment
...,...,...
44173,th,Health & Wellness
44174,their,Health & Wellness
44175,us,Health & Wellness
44176,virulenc,Health & Wellness


In [14]:
roots_df = pd.crosstab(cat_roots.root, cat_roots.categories)
roots_df

categories,Arts & Entertainment,Biographies & Memoirs,Business & Careers,Children\'s Audiobooks,Comedy & Humor,Education & Learning,Erotica,Health & Wellness,History,Home & Garden,...,Money & Finance,"Mystery, Thriller & Suspense",None,Politics & Social Sciences,"Relationships, Parenting & Personal Development",Religion & Spirituality,Romance,Science & Engineering,Science Fiction & Fantasy,Teen & Young Adult
root,,,,,,,,,,,,,,,,,,,,,
,5,64,55,14,3,3,1,47,13,1,...,12,65,2,15,42,11,41,3,119,32
$,3,20,18,1,0,1,0,13,7,0,...,4,13,0,3,7,1,5,1,16,7
abandon,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,3,0
abandoning,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
abb,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zomb,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
zon,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
zor,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
cat_root_prob_df = roots_df / cat_df.groupby('categories').size() + 0.001

In [16]:
cat_root_prob_df.History.sort_values(ascending=False).head(20)

root
th          1.0010
an          1.0010
of          0.9385
            0.8135
in          0.8135
t           0.8135
on          0.5010
from        0.5010
with        0.4385
$           0.4385
worl        0.4385
it          0.4385
for         0.3760
american    0.3760
new         0.3760
that        0.3760
history     0.3760
americ      0.3135
h           0.3135
by          0.3135
Name: History, dtype: float64

In [17]:
def predict_categories(summary):
    factors = category_prob * cat_root_prob_df[
        cat_root_prob_df.index.isin(
            lexical_roots(summary)
        )
    ].prod()
    factors /= max(factors)
    return factors.sort_values(ascending=False).head(5)

In [18]:
predict_categories("Tom Felton’s adolescence was anything but ordinary.")

categories
Biographies & Memoirs           1.000000
Mystery, Thriller & Suspense    0.001701
Science Fiction & Fantasy       0.000360
Literature & Fiction            0.000225
Romance                         0.000095
dtype: float64

In [19]:
def prediction_score(summary, category):
    prediction_table = predict_categories(summary)
    n =  prediction_table[category]
    return n/(sum(prediction_table))
prediction_score("Tom Felton’s adolescence was anything but ordinary.",'Biographies & Memoirs')

0.9976248989297929

In [20]:
predictions_df = df[['categories']].reset_index()
def predict_one(summary):
    return predict_categories(summary).index[0]
predictions_df['predicted'] = cat_df['summary'].apply(predict_one)
predictions_df

,audiobook_id,categories,predicted
0,B09RQ4HRHN,Arts & Entertainment,Arts & Entertainment
1,B09SBP7MWN,Romance,Romance
2,B09R62PV4B,"Mystery, Thriller & Suspense","Mystery, Thriller & Suspense"
3,B09VHWHJS2,Biographies & Memoirs,Biographies & Memoirs
4,1524779261,Business & Careers,Business & Careers
...,...,...,...
760,B0BK2RGQBT,Biographies & Memoirs,Biographies & Memoirs
761,B09V1X64YV,Science Fiction & Fantasy,Science Fiction & Fantasy
762,0062963708,Literature & Fiction,Literature & Fiction
763,1250752337,"Mystery, Thriller & Suspense","Mystery, Thriller & Suspense"
